In [6]:
%python --version
%pip install pandas
%pip install geopy
%pip install numpy
%pip install gurobipy

UsageError: Line magic function `%%pip` not found.


**Imports and Paths**

In [1]:
import pandas as pd
import networkx as nx
from geopy.distance import geodesic
import numpy
small_path = "./main_data/small_data/"
main_path = "./main_data/"

**Load Data**

In [ ]:
# Function -------------------------------------------------------------
def load_total_demand():

    yearly_sales = pd.read_csv(f'{main_path}vw_yearly_sales.csv')
    return yearly_sales


# Function -------------------------------------------------------------
def load_dcs(small = False):
  
    # NOTE: stateconst is a constant used to account for the different prices for the cost of living and rent in various states
    if small:
        dcs = pd.read_csv(f'{small_path}small_list_of_dcs.csv', encoding= 'unicode_escape', names=['ID', "Name","City","State","Address Line 1","Address Line 2", "Coordinates", "Area m3", "Facility Type", "StateConst"])
    else:
        dcs = pd.read_csv(f'{main_path}list_of_dcs.csv', encoding= 'unicode_escape', names=['ID', "Name","City","State","Address Line 1","Address Line 2", "Coordinates", "Area m3", "Facility Type", "StateConst"])

    dcs['Area sqft'] = round(dcs['Area m3'] * 10.7639)
    return dcs


# Function -------------------------------------------------------------
def load_dealers(small = False):
  # https://www.vw.com/app/dccsearch/vw-us/en/Find%20a%20Volkswagen%20Dealer/+/38.353354499999995/-95.3817145/3/+/+/+/+

    if small:
        dealers = pd.read_csv(f'{small_path}small_list_of_dealers.csv',names=['ID', "Name","Address Line 1","Address Line 2", "Website","Phone Number", "Coordinates"])
    else:
        # FULL DATASET
        dealers= pd.read_csv(f'{main_path}list_of_dealers.csv',names=['ID', "Name","Address Line 1","Address Line 2", "Website","Phone Number", "Coordinates"])
    
    return dealers

# Function -------------------------------------------------------------
def load_plants():
    
    plants= pd.read_csv(f'{main_path}list_of_vw_plants_and_products.csv', encoding= 'unicode_escape',names=['ID', "City","Country", "Model","Coordinates","Current/Former plant"])
    plants_usa = plants.loc[plants["Country"]=="USA"]

    return plants_usa


# Function -------------------------------------------------------------
def load_dealer_demands():

    def single_sample():
        # 4 samples
        dealers= pd.read_csv(f'{small_path}small_list_of_demand_data.csv',names=['ID',"Name","Rating","Proportion","na"])
        del dealers[dealers.columns[-1]]
        return dealers

    def full_data():
        # FULL DATASET
        dealers= pd.read_csv(f'{main_path}list_of_demand_data.csv',names=['ID',"Name","Rating","Proportion","na", "na2", "na3"])
        del dealers[dealers.columns[-1]]
        del dealers[dealers.columns[-1]]
        del dealers[dealers.columns[-1]]
        return dealers

    # either call single_sample or full_data
    dealers = full_data() 

    return dealers

**Location Work**

In [ ]:
# NOTE: Distances are in miles (mi).
# NOTE: All areas are in square-feet (sqft)

# Function -------------------------------------------------------------
def dc_to_dealer_distances(dcs, dealers): 
    
    # using geodesic distance, may want to add 15% or something onto distance
    distance_dc_to_dealer = numpy.zeros((len(dcs), len(dealers)))

    for dc_id, dc in dcs.iterrows():

      for dealer_id, dealer in dealers.iterrows():

        point1, point2 = dc["Coordinates"].split(","), dealer["Coordinates"].split(",")
        distance = geodesic(point1, point2).miles
        dc_id = dc["ID"]
        dealer_id = dealer["ID"]
        distance_dc_to_dealer[dc_id][dealer_id] = distance

    distance_dc_to_dealer_df = pd.DataFrame(distance_dc_to_dealer)
    
    return distance_dc_to_dealer_df


# Function -------------------------------------------------------------
def plant_to_dc_distances(plants, dcs):
  
  # get coordinates of the 1 usa plant
  plant_coordinates = plants['Coordinates'].iloc[0].split(",")
  distance_plants_to_dcs = [] # since we only have 1 usa plant we can just make everything from there (very convenient)
  
  for dcs_id, dc in dcs.iterrows():

    point1 = plant_coordinates
    point2 = dc["Coordinates"].split(",")
    distance = geodesic(point1, point2).miles
    dc_id = dc["ID"]
    distance_plants_to_dcs.append(distance)
  
  return distance_plants_to_dcs

# Get distances from plant-to-DC and DC-to-dealer
dcs_to_dealers = dc_to_dealer_distances(dcs, dealers)
plants_to_dcs = plant_to_dc_distances(plants, dcs)

**Calculate Demand**

In [ ]:
# Function -------------------------------------------------------------
def get_dealer_demands(year):
  
  dealers = load_dealers()
  dealer_demands = load_dealer_demands()
  # demand for each specific car type
  total_demands = load_total_demand() 

  def get_demands(year):
    year_demands = total_demands.loc[total_demands['year'] == year]
    year_demands = year_demands[["Atlas","Cross Sport","ID4","Passat"]]
    return year_demands

  year_demands = get_demands(2021)
  year_demands = year_demands.values.tolist()[0]

  proportion_sum = dealer_demands["Proportion"].sum()

  dealer_demands [["Ratio of sales"]] = None
  dealer_demands["Ratio of sales"] = dealer_demands["Proportion"]/proportion_sum

  # with the sales proportion, get the general sales for each location
  temp_ratio_col = pd.DataFrame(dealer_demands["Ratio of sales"])

  temp = []

  for x in list(dealer_demands["Ratio of sales"]):

    temp1=[]

    for y in list(year_demands):
      
      temp1.append(round(x*y))

    temp.append(temp1)

  sales = pd.DataFrame(temp, columns=["Atlas","Cross Sport","ID4","Passat"])
  
  return sales

In [ ]:
sales = get_dealer_demands(2021)

**Calculating Costs**

In [ ]:
# NOTE: WE NEED TO BE CAREFUL ABOUT TYPE 2 FACILITIES:

# Function -------------------------------------------------------------
def calculate_dc_cost():

    # Function -------------------------------------------------------------
    def calculate_dc_cost_monthly(id, state, facility_type, areasqft):
        #2$ baseline
        cost = 1.5*areasqft 

        # https://glcdistribution.com/customer-resources/tools/warehousing-insurance-calculator/ 1 million dollar standard
        insurance = 3850 

        # https://www.payscale.com/research/CA/Job=Warehouse_Material_Handler/Hourly_Rate
        employee_salaries = (areasqft/2000)*20*(40*4) 

        # pay 1 dollar per sqft for utility
        utilities = areasqft

        # $600 – $950 per month to lease a $50,000 forklift.
        # https://discord.com/channels/260272353118912522/1093655130076893284/1129635416409133076 1 forklift services 5000 sqft
        forklift_cost =  areasqft/5000  *800

        # https://discord.com/channels/260272353118912522/1093655130076893284/1129639090942398504
        price_it_costs_to_build_warehouse_sqft = 100*areasqft
        property_tax = 6/1000*price_it_costs_to_build_warehouse_sqft

        # 1000 fixed cost monitoring + 20$/h * 40h/week * 28 people
        security = 1000 + (5 * 8 * 28*20) 

        if facility_type == 1:
            ft = 1
        else:
            ft = 1.2

        # 508 is the price of 100 big macs and state is the price of 100 big macs in that state
        price = (cost + insurance + employee_salaries + utilities + forklift_cost + security + property_tax)* (state/508)*ft
        
        return price

    prices = []

    for i in range(len(dcs.index)):
        moneys = calculate_dc_cost_monthly(float(dcs.loc[i,["ID"]]), float(dcs.loc[i,["StateConst"]]), float(dcs.loc[i,["Facility Type"]]),float(dcs.loc[i,["Area sqft"]] ))
        prices.append(moneys*12)
    
    return prices

# Function -------------------------------------------------------------
def calculate_dc_shipment_capacity():
  # This is: C_i

  dcs = load_dcs()
  dcs = dcs[["Area sqft",'Facility Type']]

  # The largest car to consider is the Cross Sport
  # It has dimensions: 4,966 mm L x 1,990 mm W x 1,723 mm H
  # we can round this up to: 17 ft length, 7 foot wide, 6 foot tall
  # add about 2 ft to everything as a buffer
  # new measurement is about 18 ft length, 8ft wide, 8 foot tall
  # so, we can predict that 2 car takes up 144/2, each car takes 72 sq foot

  # from https://discord.com/channels/260272353118912522/1093655130076893284/1131271495004459018,
  # we see that 60-80% of DCs are for storage. Let's assume 60%
  vehicle_capacity = dcs["Area sqft"]*0.6 /72

  # from https://discord.com/channels/260272353118912522/1093655130076893284/1131266537869820075
  # we see that most vehicles stay in the dc a couple of weeks (take 2 weeks)
  # we can frame this as: All vehicles in the DCS arrive at time 0, and after 2 weeks all are released
  annual_throughput = round(vehicle_capacity) * (52/2)

  return (list(annual_throughput))


**Determining the Cost of Transportation**

In [ ]:
## THE FUNCTION OF SHIPPING TRUCKS CAN BE APPROXIMATED BY A PIECEWISE FUNCTION
# https://www.desmos.com/calculator/ipxbog2uyr
# constructed from the data here: https://www.marketwatch.com/guides/car-shipping/truck-shipping/#:~:text=According%20to%20quotes%20we%20received,when%20using%20an%20enclosed%20carrier.

# Function -------------------------------------------------------------
def fixed_cost_shipment(): 
  # This is: T_1
  return 186 # cost to drive 0 miles


# Function -------------------------------------------------------------
def calculate_variable_cost_shipment(mileage): 
  # This is: T_2
  # if mileage < 1135:
  #   return mileage * (551/535)
  # return mileage * (0.43)
  return (0.43)

  # TODO: WE ARE ASSUMING FOR SIMPLICYT SAKE 0.43$/MILE AND ARE NOT CONSIDERING THE OTHER FUNCTION FOR MODELLING RIGHT NOW

**Time of Model Building**

In [ ]:
## The modelling?
import gurobipy as gp
from gurobipy import GRB

# Create environment with WLS license
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', '47fa1c45-5377-4f93-87d4-3da7da2b6955')
e.setParam('WLSSECRET', '7dfbddcb-7333-4ec7-927d-b459cffd9565')
e.setParam('LICENSEID', 868415)
e.start()

**Define Model Sets**

In [ ]:
# Get all plants, DCs, and Dealers
plants = load_plants()
dcs = load_dcs()
dealers = load_dealers()

# Get all distances, demands, and costs
dcs_to_dealers = dc_to_dealer_distances(dcs, dealers)
plants_to_dcs = plant_to_dc_distances(plants, dcs)
dealer_demands = get_dealer_demands(2021) ## car demand for each dealership
dc_annual_cost = calculate_dc_cost()

In [ ]:
# define sets
# set S
plants_set = [0]
# set I
dcs_set = list(dcs["ID"])
# set J
dealers_set = list(dealers["ID"])
# we define 0 to 3, corresponding to Atlas, CrossSport, ID4 and Passat vehicles (set V)
vehicle_set = [0,1,2,3]
# set P
facility_type_set = [0, 1]

In [ ]:
# Parameter prep - we change the dataframes to arrays of sorts, which we can use indexes and such in the formulation
d_jv = dealer_demands.values
m_ij = dcs_to_dealers.values
c_si = plants_to_dcs
# load factor is 1
L_j = 1 
T_1 = fixed_cost_shipment()
T_2 = calculate_variable_cost_shipment("dummy_value") # see code comments on why we do this as a temp hack for a preliminary model
f_ip = dc_annual_cost
C_i = calculate_dc_shipment_capacity()

In [ ]:
# Create Model
m = gp.Model(env=e)

# CREATE THE DECISION VARIABLES HERE
x1_siv = m.addVars(plants_set, dcs_set, vehicle_set, vtype=GRB.INTEGER)
x2_ijv = m.addVars(dcs_set, dealers_set, vehicle_set, vtype=GRB.INTEGER )
y_ip = m.addVars(dcs_set, facility_type_set, vtype=GRB.BINARY)